In [58]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [59]:
CFP24=pd.read_csv('BC_CFP24.csv',index_col='Sl.')

In [106]:
# Define mapping dictionary for regions
region_mapping = {
    'Vancouver Island': 'Capital',
    'South Interior West': 'Thompson-Nicola',
    'Central Interior': 'Cariboo',
    'North Coast': 'Skeena-QueenCharlotte',
    'Peace': 'PeaceRiver'
}

# Create a new column with matched names
CFP24['GADM_Region'] = CFP24['Regional system'].map(region_mapping)
CFP24['Resource_type'] = 'wind'
CFP24.to_csv('BC_CFP24.csv',index=True)

In [ ]:
# Perform the merge
merged_data = pd.merge(CFP24, boundary, left_on="GADM_Region", right_on="Region", how="inner")

# Convert merged DataFrame to a GeoDataFrame (assuming 'region_geom' is valid geometry)
geo_merged = gpd.GeoDataFrame(merged_data, geometry="geometry")

geo_merged


In [ ]:
# Create an interactive map
geo_merged.explore(
    column='Region',  # The column to color by
    markersize='Project size in megawatts (MW)',  # The column determining marker size
    legend=True,  # Show a legend
    cmap='viridis'  # Colormap for visualization
)


In [84]:
rod=pd.read_excel('rod_202409.xlsm',sheet_name="Resource Options",skiprows=1)

In [85]:
# Convert to float to ensure proper type
rod['Latitude'] = pd.to_numeric(rod['Latitude'], errors='coerce')
rod['Longitude'] = pd.to_numeric(rod['Longitude'], errors='coerce')

# Drop rows with invalid coordinates
rod = rod.dropna(subset=['Latitude', 'Longitude'])

# Create geometry column with correct longitude, latitude order
rod['geometry'] = rod.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

# Convert to GeoDataFrame
rod_gdf = gpd.GeoDataFrame(rod, geometry='geometry', crs='EPSG:4326')
rod_gdf.to_pickle('ROD_processed.pkl')

In [86]:
# Create a mask for rows with 'Solar' or 'Onshore Wind' in 'Resource Type'
VREs_mask = ( (rod_gdf['Resource Type'] == 'Onshore Wind'))# |rod_gdf['Resource Type'] == 'Solar') 
# Apply the mask to filter the GeoDataFrame
filtered_rod_gdf = rod_gdf[VREs_mask]

# Display the filtered GeoDataFrame
filtered_rod_gdf=filtered_rod_gdf.head(50)
filtered_rod_gdf['Rank'] = range(1, len(filtered_rod_gdf) + 1)

In [87]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis
store=f"../data/store/resources_BC.h5"
datahandler=DataHandler(store)
boundary=datahandler.from_store('boundary')

In [ ]:
clusters_wind.head(10).explore('LCOE')

In [96]:
cells=datahandler.from_store('cells')
cells=cells[cells['potential_capacity_wind']>7]
clusters_wind=datahandler.from_store('clusters/wind')#.head(5)
clusters_solar=datahandler.from_store('clusters/solar').head(2)

In [ ]:
import folium

# Initialize a Folium map centered at the mean coordinates of the data
map_center = [rod_gdf.geometry.y.mean(), filtered_rod_gdf.geometry.x.mean()]
m = folium.Map(location=map_center, zoom_start=6, tiles="CartoDB positron")  # Clean, modern map tiles

# Create FeatureGroups for each type of layer
boundary_group = folium.FeatureGroup(name="Boundary Layer")
wind_cells_group = folium.FeatureGroup(name="Wind Cells")
wind_clusters_group = folium.FeatureGroup(name="Wind Clusters")
solar_clusters_group = folium.FeatureGroup(name="Solar Clusters")
points_group = folium.FeatureGroup(name="Project Points")

# Add the boundary GeoDataFrame as a GeoJson layer
boundary_layer = folium.GeoJson(
    boundary,  # This is your boundary GeoDataFrame
    style_function=lambda x: {
        'fillColor': 'None',  # No fill color for the boundary
        'color': 'slategray',  # Border color for boundary regions
        'weight': .5,
        'ls':":",
        'fillOpacity': 0.2  # Transparency level
    }
)
boundary_group.add_child(boundary_layer)

# Add the MultiPolygon geometries (Wind Cells) directly to the map as GeoJson
for _, row in cells.iterrows():
    folium.GeoJson(
        row.geometry,  # The MultiPolygon geometry
        style_function=lambda x: {
            'fillColor': 'violet',  # Color of the polygon
            'color': 'violet',  # Border color of the polygon
            'weight': .1,  # Border thickness
            'fillOpacity': 0.1  # Fill opacity of the polygon
        },
        tooltip=folium.Tooltip(f"{row['potential_capacity_wind']}<br>"
                               f"Region: {row['Region']}<br>"
                               f"LCOE ($/MWh): {row['LCOE_wind']}<br>"
                               f"Windspeed_GWA: {round(row['windspeed_gwa'], 2)} m/s")  # Show info on hover
    ).add_to(wind_cells_group)

# Add the MultiPolygon geometries (Wind Clusters) directly to the map as GeoJson
for _, row in clusters_wind.iterrows():
    folium.GeoJson(
        row.geometry,  # The MultiPolygon geometry
        style_function=lambda x: {
            'fillColor': 'violet',  # Color of the polygon
            'color': 'violet',  # Border color of the polygon
            'weight': 0.5,  # Border thickness
            'fillOpacity': 0.5  # Fill opacity of the polygon
        },
        tooltip=folium.Tooltip(f"{round(row['potential_capacity']/1E3,2)} GW <br>"
                               f"Region: {row['Region']}<br>"
                               f"Rank: {row['Rank']}<br>"
                               f"LCOE ($/MWh): {round(row['LCOE'],2)}<br>"
                               )  # Show info on hover
    ).add_to(wind_clusters_group)

# Add the MultiPolygon geometries (Solar Clusters) directly to the map as GeoJson
for _, row in clusters_solar.iterrows():
    folium.GeoJson(
        row.geometry,  # The MultiPolygon geometry
        style_function=lambda x: {
            'fillColor': 'lawngreen',  # Color of the polygon
            'color': 'lawngreen',  # Border color of the polygon
            'weight': 1,  # Border thickness
            'fillOpacity': 0.3  # Fill opacity of the polygon
        },
        tooltip=folium.Tooltip(f"{round(row['potential_capacity']/1E3,2)} GW <br>"
                               f"Region: {row['Region']}<br>"
                               f"Rank: {row['Rank']}<br>"
                               f"LCOE ($/MWh): {round(row['LCOE'],2)}<br>"
                               )  # Show info on hover
    ).add_to(solar_clusters_group)

# Add points to the map without clustering
for _, row in filtered_rod_gdf.iterrows():
    folium.CircleMarker(
        location=(row.geometry.y, row.geometry.x),  # Lat, Long
        radius=row['Installed Capacity (MW)'] * 0.04,  # Scale marker size
        weight=.5,  # Thickness of the border
        color='orange' if row['Resource Type'] == 'Solar' else 'navy',  # Different colors for resource types
        fill=True,
        fill_opacity=0.3,
        popup=folium.Popup(f"<b>{row['Project Name']}</b><br>"
                           f"<b>{row['Resource Type']}</b><br>"
                           f"<strong>Rank: {int(row['Rank'])}</strong> <br>"
                           f"<strong>TUEC: {round(row['Total Adj UEC1 ($/MWh)'],2)}$/MWh</strong> <br>"
                           f"<strong>{int(row['Installed Capacity (MW)'])} MW</strong><br>"
                           f"<i>{row['Region']}</i>", max_width=250),  # Clean and readable popup
    ).add_to(points_group)

# Add all feature groups to the map
m.add_child(boundary_group)
m.add_child(wind_cells_group)
m.add_child(wind_clusters_group)
m.add_child(solar_clusters_group)
m.add_child(points_group)

# Add a sleek legend manually with modern style
legend_html = """
<div style="position: fixed; bottom: 30px; left: 30px; width: 250px; height: 160px; 
background-color: rgba(255, 255, 255, 0.7); border:2px solid #ccc; z-index:9999; font-size:14px; 
border-radius: 10px; padding: 10px;">
  <b>Legend</b><br>
  <i style="color:lawngreen;">●</i> Solar<br>
  <i style="color:navy;">●</i> Onshore Wind<br>
  <i style="color:violet;">●</i> Wind Clusters<br>
  <i style="color:lawngreen;">●</i> Solar Clusters<br>
  <i style="color:slategrey;">■</i> Boundary Region<br>  <!-- For boundary regions -->
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Add layer control to toggle visibility of the layers
folium.LayerControl().add_to(m)

# Save and display the map
m.save('ROD_2024_with_boundary.html')
m
